In [454]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import geopandas as gpd

## Cleaning the racial data by county level

Reading the csv file with racial demographics of counties in Georgia taken from the US Census Bureau. 

In [455]:
# Read csv file to dataframe
race_county =  pd.read_csv('data/race_county_data/original_georgia_race_data.csv', skiprows = 1)

pd.options.display.width=None
pd.set_option('max_row', None)
pd.set_option('display.max_rows', race_county.shape[0] + 1)
pd.set_option('display.expand_frame_repr', False)

In [456]:
# View the column names and the number of rows and columns 
print(race_county.columns)
print(race_county.shape)

Index(['id', 'Geographic Area Name', ' !!Total:',
       ' !!Total:!!Hispanic or Latino', ' !!Total:!!Not Hispanic or Latino:',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:!!American Indian and Alaska Native alone',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone',
       ' !!Total:!!Not Hispanic or Latino:!!Population of one race:!!Some Other Race alone',
       ' !!Total:!!Not Hispanic or Latino:!!Population of two or more races:',
       ' !!Total:!!Not Hispanic or Latino:!!Population of two or more races:!!Population of two races:',
       ' !!Total:!!Not Hi

We will combine the data on the number of people that identify as more than one race as 'Mixed'. We also will combine the columns with the data on the number of people that identify their race as 'American Indian and Alaska Native alone, Native Hawaiian' and 'Other Pacific Islander alone' and 'Some Other Race alone' into an 'Others' column. There's also a weird space at the start of each column name so we can remove that.

In [457]:
# Remove the space at the start of each column
race_county.columns = race_county.columns.map(str)
race_county.columns = race_county.columns.str.lstrip()

# Create the column 'Mixed'
race_county.rename(columns={'!!Total:!!Not Hispanic or Latino:!!Population of two or more races:':'Mixed'}, inplace = True)

# Create the column 'Others'
race_county['Others'] = race_county['!!Total:!!Not Hispanic or Latino:!!Population of one race:!!American Indian and Alaska Native alone'] + race_county['!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone'] + race_county['!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Some Other Race alone']



Then we create a new dataframe with only the columns we want: 'Total', 'White', 'Black', 'Asian' and the previously created columns of 'Others' and 'Mixed. We will also rename the columns accordingly.

In [458]:
# Create a new dataframe with the desired columns
race_county = race_county[['Geographic Area Name', 'id', '!!Total:', '!!Total:!!Hispanic or Latino', '!!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone', '!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone','!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone', 'Mixed', 'Others' ]].copy()

# Rename the columns
race_county.columns = ['Area Name', 'id', 'Total', 'Hispanic', 'White', 'Black', 'Asian', 'Mixed', 'Others']

In [459]:
race_county.head(20)


,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others
0,"Appling County, Georgia",0500000US13001,18444,1825,12674,3339,123,417,66
1,"Atkinson County, Georgia",0500000US13003,8286,2048,4801,1208,12,167,50
2,"Bacon County, Georgia",0500000US13005,11140,875,8103,1747,40,335,40
3,"Baker County, Georgia",0500000US13007,2876,143,1514,1128,18,70,3
4,"Baldwin County, Georgia",0500000US13009,43799,1139,22432,18318,599,1027,284
5,"Banks County, Georgia",0500000US13011,18035,1164,15578,394,189,620,90
6,"Barrow County, Georgia",0500000US13013,83505,10560,55582,10141,3233,3383,606
7,"Bartow County, Georgia",0500000US13015,108901,10751,80159,11309,1169,4753,760
8,"Ben Hill County, Georgia",0500000US13017,17194,1054,9219,6222,116,478,105
9,"Berrien County, Georgia",0500000US13019,18160,1045,14396,1934,80,611,94


Creating rows with density. 

In [460]:
for ind, row in race_county.iterrows():
  race_county.loc[ind,"Population Density:Hispanic"]= row ['Hispanic']/row['Total']
  race_county.loc[ind,"Population Density: White"] = row ['White']/row['Total']
  race_county.loc[ind,"Population Density: Black"]= row['Black']/row['Total']
  race_county.loc[ind,"Population Density: Asian"]= row ['Asian']/row['Total']
  race_county.loc[ind,"Population Density: Mixed"]=row['Mixed']/row["Total"]
  race_county.loc[ind,"Population Density: Others"]=row["Others"]/row["Total"]

In [461]:
race_county.tail()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density:Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others
154,"Whitfield County, Georgia",0500000US13313,102864,36916,57875,3553,1394,2621,505,0.358882,0.562636,0.034541,0.013552,0.025480,0.004909
155,"Wilcox County, Georgia",0500000US13315,8766,272,5185,3096,49,149,15,0.031029,0.591490,0.353183,0.005590,0.016997,0.001711
156,"Wilkes County, Georgia",0500000US13317,9565,399,4952,3838,59,277,40,0.041715,0.517721,0.401255,0.006168,0.028960,0.004182
157,"Wilkinson County, Georgia",0500000US13319,8877,239,5110,3163,22,297,46,0.026924,0.575645,0.356314,0.002478,0.033457,0.005182
158,"Worth County, Georgia",0500000US13321,20784,381,14427,5255,87,524,110,0.018331,0.694140,0.252839,0.004186,0.025212,0.005293


Downloading the dataframes into new files.

In [462]:
race_county.to_csv('data/race_county_data/cleaned_georgia_race_county.csv')


# Cleaning the racial data by precinct level

Reading the csv file with racial demographics of precincts in Georgia taken from the US Census Bureau. The steps for cleaning the data are the same as those for the county data, except that we also have to remove the top row as that shows the racial demographics of just Georgia.

In [463]:
# Read csv file to dataframe
race_precinct =  pd.read_csv('data/race_precinct_data/original_georgia_race_data.csv', skiprows = 1)

race_precinct.columns = race_precinct.columns.str.lstrip()

race_precinct.columns = race_precinct.columns.map(str)

race_precinct.rename(columns={'!!Total:!!Not Hispanic or Latino:!!Population of two or more races:':'Mixed'}, inplace = True)

race_precinct['Others'] = race_precinct['!!Total:!!Not Hispanic or Latino:!!Population of one race:!!American Indian and Alaska Native alone'] + race_precinct['!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Native Hawaiian and Other Pacific Islander alone'] + race_precinct['!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Some Other Race alone']

# Create a new dataframe with the desired columns
race_precinct = race_precinct[['Geographic Area Name', 'id', '!!Total:', '!!Total:!!Hispanic or Latino', '!!Total:!!Not Hispanic or Latino:!!Population of one race:!!White alone', '!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Black or African American alone','!!Total:!!Not Hispanic or Latino:!!Population of one race:!!Asian alone', 'Mixed', 'Others' ]].copy()

# Rename the columns
race_precinct.columns = ['Area Name', 'id', 'Total', 'Hispanic', 'White', 'Black', 'Asian', 'Mixed', 'Others']

# Remove the top row with the racial demographic of Georgia
race_precinct.drop(0, inplace = True)

race_precinct.to_csv('data/race_precinct_data/cleaned_georgia_race_precinct.csv')


Now that the columns have been cleaned, we need to add columns with the racial densities of each precinct.

In [464]:
race_precinct =  pd.read_csv('data/race_precinct_data/cleaned_georgia_race_precinct.csv', index_col=0)

race_precinct.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others
1,"2, Appling County, Georgia",7000000US13001000002,3563,403,1215,1838,10,81,16
2,"1B, Appling County, Georgia",7000000US1300100001B,1834,76,1575,109,3,63,8
3,"1C, Appling County, Georgia",7000000US1300100001C,1538,116,1242,150,5,25,0
4,"3C, Appling County, Georgia",7000000US1300100003C,2515,263,1528,608,39,69,8
5,"4B, Appling County, Georgia",7000000US1300100004B,1321,62,1147,59,5,43,5


After doing this step, we see that there is an error 'dividing by 0'

In [ ]:
for ind, row in race_precinct.iterrows():
  race_precinct.loc[ind,"Population Density: Hispanic"]= row ['Hispanic']/row['Total']
  race_precinct.loc[ind,"Population Density: White"] = row ['White']/row['Total']
  race_precinct.loc[ind,"Population Density: Black"]= row['Black']/row['Total']
  race_precinct.loc[ind,"Population Density: Asian"]= row ['Asian']/row['Total']
  race_precinct.loc[ind,"Population Density: Mixed"]=row['Mixed']/row["Total"]
  race_precinct.loc[ind,"Population Density: Others"]=row["Others"]/row["Total"]

Thus we have to see which precincts have a population total of 0.

In [465]:
race_precinct.loc[race_precinct['Total']==0]

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others
142,"FORT STEWART, Bryan County, Georgia",7000000US13029FTSTEW,0,0,0,0,0,0,0
334,"FORT PULASKI MON, Chatham County, Georgia",7000000US130510XFTPU,0,0,0,0,0,0,0
1316,"08F2, Fulton County, Georgia",7000000US131210008F2,0,0,0,0,0,0,0
1325,"12E2, Fulton County, Georgia",7000000US131210012E2,0,0,0,0,0,0,0
1399,"AP01E, Fulton County, Georgia",7000000US131210AP01E,0,0,0,0,0,0,0
1412,"AP12D, Fulton County, Georgia",7000000US131210AP12D,0,0,0,0,0,0,0
1418,"CP04A, Fulton County, Georgia",7000000US131210CP04A,0,0,0,0,0,0,0
1422,"CP053, Fulton County, Georgia",7000000US131210CP053,0,0,0,0,0,0,0
1489,"SC07B, Fulton County, Georgia",7000000US131210SC07B,0,0,0,0,0,0,0
1491,"SC08A, Fulton County, Georgia",7000000US131210SC08A,0,0,0,0,0,0,0


So that these rows are skipped, we replace the rows with 0 with NaN.

In [466]:
race_precinct.replace(0, np.nan, inplace=True)

In [467]:
for ind, row in race_precinct.iterrows():
  race_precinct.loc[ind,"Population Density: Hispanic"]= row ['Hispanic']/row['Total']
  race_precinct.loc[ind,"Population Density: White"] = row ['White']/row['Total']
  race_precinct.loc[ind,"Population Density: Black"]= row['Black']/row['Total']
  race_precinct.loc[ind,"Population Density: Asian"]= row ['Asian']/row['Total']
  race_precinct.loc[ind,"Population Density: Mixed"]=row['Mixed']/row["Total"]
  race_precinct.loc[ind,"Population Density: Others"]=row["Others"]/row["Total"]

Then we replace NaN values with 0 so that on the map they show that the population is 0.

In [468]:
race_precinct.replace(np.nan, 0, inplace=True)

race_precinct.loc[race_precinct['Total']==0]

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density: Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others
142,"FORT STEWART, Bryan County, Georgia",7000000US13029FTSTEW,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334,"FORT PULASKI MON, Chatham County, Georgia",7000000US130510XFTPU,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1316,"08F2, Fulton County, Georgia",7000000US131210008F2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1325,"12E2, Fulton County, Georgia",7000000US131210012E2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1399,"AP01E, Fulton County, Georgia",7000000US131210AP01E,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1412,"AP12D, Fulton County, Georgia",7000000US131210AP12D,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1418,"CP04A, Fulton County, Georgia",7000000US131210CP04A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1422,"CP053, Fulton County, Georgia",7000000US131210CP053,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1489,"SC07B, Fulton County, Georgia",7000000US131210SC07B,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1491,"SC08A, Fulton County, Georgia",7000000US131210SC08A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [469]:
race_precinct.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density: Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others
1,"2, Appling County, Georgia",7000000US13001000002,3563.0,403.0,1215.0,1838.0,10.0,81.0,16.0,0.113107,0.341005,0.515857,0.002807,0.022734,0.004491
2,"1B, Appling County, Georgia",7000000US1300100001B,1834.0,76.0,1575.0,109.0,3.0,63.0,8.0,0.041439,0.858779,0.059433,0.001636,0.034351,0.004362
3,"1C, Appling County, Georgia",7000000US1300100001C,1538.0,116.0,1242.0,150.0,5.0,25.0,0.0,0.075423,0.807542,0.097529,0.003251,0.016255,0.000000
4,"3C, Appling County, Georgia",7000000US1300100003C,2515.0,263.0,1528.0,608.0,39.0,69.0,8.0,0.104573,0.607555,0.241750,0.015507,0.027435,0.003181
5,"4B, Appling County, Georgia",7000000US1300100004B,1321.0,62.0,1147.0,59.0,5.0,43.0,5.0,0.046934,0.868282,0.044663,0.003785,0.032551,0.003785


In [470]:
race_precinct.to_csv('data/race_precinct_data/cleaned_georgia_race_precinct_densities.csv')

# Preparing the polling site data for county level for 2016, 2018 and 2020 data

As we will be analysing the change in the number of polling sites in counties, the polling site data for 2016, 2018 and 2020 will be prepared. The raw data consists of the addresses of polling sites in each county.

In [471]:
polling_2016_path = "data/polling_site_data/2016_original.csv"
polling_2018_path = "data/polling_site_data/2018_original.csv"
polling_2020_path = "data/polling_site_data/2020_original.csv"

polling_2016 = pd.read_csv(polling_2016_path)
polling_2018 = pd.read_csv(polling_2018_path)
polling_2020 = pd.read_csv(polling_2020_path)

Each polling site address is in a separate row and has a county name. This step prepares the data by counting how many times a county is repeated in the 'county_name' column. This represents the number of polling sites in that county.

In [472]:
polling_2016 = polling_2016['county_name'].value_counts().rename_axis('County').reset_index(name='2016 Quantity')
polling_2018 = polling_2018['county_name'].value_counts().rename_axis('County').reset_index(name='2018 Quantity')
polling_2020 = polling_2020['county_name'].value_counts().rename_axis('County').reset_index(name='2020 Quantity')

polling_2016['County'] = polling_2016['County'].str.title()
polling_2018['County'] = polling_2018['County'].str.title()
polling_2020['County'] = polling_2020['County'].str.title()

In [473]:
polling = polling_2016.merge(polling_2018, how='left', left_on="County", right_on="County")
polling = polling.merge(polling_2020, how='left', left_on="County", right_on="County")


In [474]:
polling.head()

,County,2016 Quantity,2018 Quantity,2020 Quantity
0,Fulton,364,382,394
1,Dekalb,191,191,193
2,Gwinnett,156,156,156
3,Cobb,145,141,145
4,Chatham,89,90,92


In [475]:
polling.to_csv('data/polling_site_data/polling_sites_in_years.csv')

# Preparing polling site densities for 2020

As part of the methodology, we want to see the polling site densities for each county in 2020. Thus, we will prepare a dataset with the racial demographics of each county and the density of polling sites in that county.

In [476]:
polling_2020 =  pd.read_csv('data/polling_site_data/polling_sites_in_years.csv', index_col = 0)
polling_2020 = polling_2020[['County', '2020 Quantity']]

In [477]:
polling_2020.head()

,County,2020 Quantity
0,Fulton,394
1,Dekalb,193
2,Gwinnett,156
3,Cobb,145
4,Chatham,92


In [478]:
print(polling_2020.shape)

(159, 2)


The cleaned dataset with the race data has the county names in the format with 'County, Georgia' at the end. The polling site dataset has the county names without 'County, Georgia', so the 'Area Name' column of the race data has to be edited. Since we are merging, it is also a good practice to standardise capitalisations.

In [479]:
race_county =  pd.read_csv('data/race_county_data/cleaned_georgia_race_county.csv', index_col = 0)

race_county['Area Name'] = race_county['Area Name'].str.replace(' County, Georgia', '')
race_county['Area Name'] = race_county['Area Name'].str.title()

race_county.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density:Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others
0,Appling,0500000US13001,18444,1825,12674,3339,123,417,66,0.098948,0.687161,0.181034,0.006669,0.022609,0.003578
1,Atkinson,0500000US13003,8286,2048,4801,1208,12,167,50,0.247164,0.579411,0.145788,0.001448,0.020154,0.006034
2,Bacon,0500000US13005,11140,875,8103,1747,40,335,40,0.078546,0.727379,0.156822,0.003591,0.030072,0.003591
3,Baker,0500000US13007,2876,143,1514,1128,18,70,3,0.049722,0.526426,0.392211,0.006259,0.024339,0.001043
4,Baldwin,0500000US13009,43799,1139,22432,18318,599,1027,284,0.026005,0.512158,0.418229,0.013676,0.023448,0.006484


In [480]:
total_population = race_county[['Area Name', 'Total']].copy()
total_population.rename(columns={'Area Name': 'County', 'Total': 'Total Population'}, inplace=True)


In [481]:
total_population.head()

,County,Total Population
0,Appling,18444
1,Atkinson,8286
2,Bacon,11140
3,Baker,2876
4,Baldwin,43799


In [482]:
print(total_population.shape)

(159, 2)


In [483]:
polling_2020 = polling_2020.merge(total_population, on='County', how='inner')
polling_2020.head()

,County,2020 Quantity,Total Population
0,Fulton,394,1066710
1,Dekalb,193,764382
2,Gwinnett,156,957062
3,Cobb,145,766149
4,Chatham,92,295291


In [484]:
print(polling_2020.shape)

(159, 3)


In [485]:
polling_2020['Polling Site Density'] = polling_2020['2020 Quantity']/polling_2020['Total Population']

In [486]:
polling_2020.head()

,County,2020 Quantity,Total Population,Polling Site Density
0,Fulton,394,1066710,0.000369
1,Dekalb,193,764382,0.000252
2,Gwinnett,156,957062,0.000163
3,Cobb,145,766149,0.000189
4,Chatham,92,295291,0.000312


In [487]:
print(polling_2020.shape)

(159, 4)


In [488]:
polling_2020.to_csv('data/polling_site_data/2020_clean_county_densities.csv')

# Preparing the polling site data for precinct level

A precinct either has a polling site or it doesn't. To analyse the correlation between the racial demographic of a precinct and whether it has a polling site, data with the precincts with polling sites in the 2020 US Election was prepared. The raw data consists of the addresses of polling sites in each county in 2020. As we will need to map the polling site data to the race data, the race dataset for counties is also read as a CSV.

In [489]:
polling_site_precinct_2020 =  pd.read_csv('data/polling_site_data/2020_original.csv')
polling_site_precinct_2020.columns = polling_site_precinct_2020.columns.map(str)
polling_site_precinct_2020['county_name'] = polling_site_precinct_2020['county_name'].str.title()

race_county =  pd.read_csv('data/race_county_data/cleaned_georgia_race_county.csv', index_col = 0)
race_county.columns = race_county.columns.map(str)

We will also see the number of rows in the polling site data so we can check at the end of the data preparation that no rows were deleted.

In [490]:
print(polling_site_precinct_2020.shape)

(2678, 15)


Firstly, we see that we don't directly know which precinct each line of data is referring to as it only includes the county name and precinct id. The race data, however, has the id of each county.

In [491]:
polling_site_precinct_2020.head()

,election_date,state,county_name,jurisdiction,jurisdiction_type,precinct_id,precinct_name,polling_place_id,location_type,name,address,notes,source,source_date,source_notes
0,2020-11-03,GA,Appling,Appling,county,2,NaN,1012,election_day,LIONS CLUB BLDG/ AT FAIR GROUNDS,"245 INDUSTRIAL DR, BAXLEY, GA 31513",OTHER,ORR,2020-10-19,NaN
1,2020-11-03,GA,Appling,Appling,county,1C,NaN,1018,election_day,1ST ASSEMBLY OF GOD CHURCH,"3397 HATCH PKY N, BAXLEY, GA 31513",CHURCH,ORR,2020-10-19,NaN
2,2020-11-03,GA,Appling,Appling,county,1B,NaN,1001,election_day,ALTAMAHA FIRE STATION,"392 ALTAMAHA SCHOOL RD, BAXLEY, GA 31513",County Building,ORR,2020-10-19,NaN
3,2020-11-03,GA,Appling,Appling,county,4D,NaN,1006,election_day,BAX CH OF GOD/FELLOWSHIP HALL,"353 BLACKSHEAR HWY, BAXLEY, GA 31513",CHURCH,ORR,2020-10-19,NaN
4,2020-11-03,GA,Appling,Appling,county,5A,NaN,1002,election_day,BAXLEY CITY GYM,"252 W. PARKER ST., BAXLEY, GA 31513",County Building,ORR,2020-10-19,NaN


In [492]:
race_county.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density:Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others
0,"Appling County, Georgia",0500000US13001,18444,1825,12674,3339,123,417,66,0.098948,0.687161,0.181034,0.006669,0.022609,0.003578
1,"Atkinson County, Georgia",0500000US13003,8286,2048,4801,1208,12,167,50,0.247164,0.579411,0.145788,0.001448,0.020154,0.006034
2,"Bacon County, Georgia",0500000US13005,11140,875,8103,1747,40,335,40,0.078546,0.727379,0.156822,0.003591,0.030072,0.003591
3,"Baker County, Georgia",0500000US13007,2876,143,1514,1128,18,70,3,0.049722,0.526426,0.392211,0.006259,0.024339,0.001043
4,"Baldwin County, Georgia",0500000US13009,43799,1139,22432,18318,599,1027,284,0.026005,0.512158,0.418229,0.013676,0.023448,0.006484


We see that the id of each county has 14 characters and only differs in the last few characters. Since the polling site data has the county that the polling site is in and its precinct id, we can merge these two id's to create each precinct's unique id. As we saw in the precinct race data, the unique id for each precinct consists of 20 characters as shown below. It is also observed that the middle characters in the id are the same for the precincts in the same county and the last 6 characters are unique for each precinct.  

In [493]:
race_precinct = pd.read_csv('data/race_precinct_data/cleaned_georgia_race_precinct_densities.csv', index_col = 0)
race_precinct.head(20)

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density: Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others
1,"2, Appling County, Georgia",7000000US13001000002,3563.0,403.0,1215.0,1838.0,10.0,81.0,16.0,0.113107,0.341005,0.515857,0.002807,0.022734,0.004491
2,"1B, Appling County, Georgia",7000000US1300100001B,1834.0,76.0,1575.0,109.0,3.0,63.0,8.0,0.041439,0.858779,0.059433,0.001636,0.034351,0.004362
3,"1C, Appling County, Georgia",7000000US1300100001C,1538.0,116.0,1242.0,150.0,5.0,25.0,0.0,0.075423,0.807542,0.097529,0.003251,0.016255,0.000000
4,"3C, Appling County, Georgia",7000000US1300100003C,2515.0,263.0,1528.0,608.0,39.0,69.0,8.0,0.104573,0.607555,0.241750,0.015507,0.027435,0.003181
5,"4B, Appling County, Georgia",7000000US1300100004B,1321.0,62.0,1147.0,59.0,5.0,43.0,5.0,0.046934,0.868282,0.044663,0.003785,0.032551,0.003785
6,"4D, Appling County, Georgia",7000000US1300100004D,2455.0,323.0,1895.0,158.0,45.0,31.0,3.0,0.131568,0.771894,0.064358,0.018330,0.012627,0.001222
7,"5A, Appling County, Georgia",7000000US1300100005A,2229.0,395.0,1521.0,226.0,12.0,66.0,9.0,0.177210,0.682369,0.101391,0.005384,0.029610,0.004038
8,"5B, Appling County, Georgia",7000000US1300100005B,1843.0,163.0,1478.0,166.0,2.0,24.0,10.0,0.088443,0.801953,0.090071,0.001085,0.013022,0.005426
9,"3A1, Appling County, Georgia",7000000US130010003A1,1146.0,24.0,1073.0,25.0,2.0,15.0,7.0,0.020942,0.936300,0.021815,0.001745,0.013089,0.006108
10,"PEARSON COUNTY, Atkinson County, Georgia",7000000US13003000001,3121.0,748.0,2106.0,174.0,1.0,69.0,23.0,0.239667,0.674784,0.055751,0.000320,0.022108,0.007369


Isolating just the id and the name of the county:

In [494]:
race_county = race_county[['id', 'Area Name']]
race_county.head()

,id,Area Name
0,0500000US13001,"Appling County, Georgia"
1,0500000US13003,"Atkinson County, Georgia"
2,0500000US13005,"Bacon County, Georgia"
3,0500000US13007,"Baker County, Georgia"
4,0500000US13009,"Baldwin County, Georgia"


To make things clearer we extract the last 7 characters of the 'id' column. To make the race data consistent with the polling site data, we will also change the 'Area Name' column so that it just shows the county name and make the county names titles.

In [495]:
race_county['id'] = race_county['id'].str[7:]
race_county['Area Name'] = race_county['Area Name'].str.replace(' County, Georgia','')
race_county['Area Name'] = race_county['Area Name'].str.title()

In [496]:
race_county.head()

,id,Area Name
0,US13001,Appling
1,US13003,Atkinson
2,US13005,Bacon
3,US13007,Baker
4,US13009,Baldwin


We want to create a dataset with just the list of precincts with polling sites, so we merge the id of each county from above to the polling site data so that we have a dataframe with the county name, id for the county, and precinct id. 

In [497]:
merged = pd.merge(polling_site_precinct_2020, race_county, how="inner", on=None, left_on=['county_name'], right_on=['Area Name'])

polling_site_precinct_2020 = merged[['id', 'Area Name', 'precinct_id']]

polling_site_precinct_2020.head(20)

,id,Area Name,precinct_id
0,US13001,Appling,2
1,US13001,Appling,1C
2,US13001,Appling,1B
3,US13001,Appling,4D
4,US13001,Appling,5A
5,US13001,Appling,5B
6,US13001,Appling,3C
7,US13001,Appling,3A1
8,US13001,Appling,4B
9,US13003,Atkinson,0001


Checking that all rows were preserved.

In [498]:
print(polling_site_precinct_2020.shape)

(2678, 3)


In [499]:
polling_site_precinct_2020.info(verbose=True)
polling_site_precinct_2020["precinct_id"]= polling_site_precinct_2020["precinct_id"].astype(str)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2678 entries, 0 to 2677
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   id           2678 non-null   object
 1   Area Name    2678 non-null   object
 2   precinct_id  2678 non-null   object
dtypes: object(3)
memory usage: 83.7+ KB


In the race data, the id for each precinct has 19 characters with the middle characters being unique for the county and the last 6 characters being unique for each precinct. In the polling site data, not all the precinct id's are 6 characters so we will fill it with 0's so that it is 6 six characters.

In [500]:
polling_site_precinct_2020['precinct_id'] = polling_site_precinct_2020['precinct_id'].str.zfill(6)

polling_site_precinct_2020.head(20)

,id,Area Name,precinct_id
0,US13001,Appling,000002
1,US13001,Appling,00001C
2,US13001,Appling,00001B
3,US13001,Appling,00004D
4,US13001,Appling,00005A
5,US13001,Appling,00005B
6,US13001,Appling,00003C
7,US13001,Appling,0003A1
8,US13001,Appling,00004B
9,US13003,Atkinson,000001


Merging the id for county and and precinct and adding the '7000000' string that was previously taken out.

In [501]:
polling_site_precinct_2020['Geographic Id'] = polling_site_precinct_2020['id'] + polling_site_precinct_2020['precinct_id']

polling_site_precinct_2020['Geographic Id'] = '7000000' + polling_site_precinct_2020['Geographic Id'].astype(str)

In [502]:
polling_site_precinct_2020 = polling_site_precinct_2020[[ 'Geographic Id', 'Area Name']]

polling_site_precinct_2020.rename(columns = {'Geographic Id': 'Id','Area Name': 'County'}, inplace = True)

Below we have the resulting dataframe of the precincts with polling sites and the county the precinct is in.

In [503]:
polling_site_precinct_2020

,Id,County
0,7000000US13001000002,Appling
1,7000000US1300100001C,Appling
2,7000000US1300100001B,Appling
3,7000000US1300100004D,Appling
4,7000000US1300100005A,Appling
...,...,...
2673,7000000US13321000005,Worth
2674,7000000US13321000004,Worth
2675,7000000US13321000014,Worth
2676,7000000US13321000009,Worth


Saving the dataframe as a CSV for future use.

In [504]:
polling_site_precinct_2020.to_csv('data/polling_site_data/2020_clean_precincts.csv')

Now, we will need to merging the dataframe with polling sites with the dataframe with race densities to give a resulting dataframe to indicate whether that precinct has a polling site. To do this we will add a column with 0's and 1's, with 1 signalling that the precinct has a polling site and 0 signalling the precinct doesn't have a polling site. We will do this for both 2020 and 2016 data as we want to analyse the racial densities of precincts that lost or gained a polling site from 2016 to 2020. 

Firstly, the 2016 polling site data has to be prepared using the same steps used previously to prepare the 2020 data.

In [505]:
polling_site_precinct_2016 =  pd.read_csv('data/polling_site_data/2016_original.csv')
polling_site_precinct_2016.columns = polling_site_precinct_2016.columns.map(str)
polling_site_precinct_2016['county_name'] = polling_site_precinct_2016['county_name'].str.title()

race_county =  pd.read_csv('data/race_county_data/cleaned_georgia_race_county.csv', index_col = 0)
race_county.columns = race_county.columns.map(str)
race_county = race_county[['id', 'Area Name']]
race_county['id'] = race_county['id'].str[7:]
race_county['Area Name'] = race_county['Area Name'].str.replace(' County, Georgia','')
race_county['Area Name'] = race_county['Area Name'].str.title()

merged_2016 = pd.merge(polling_site_precinct_2016, race_county, how="inner", on=None, left_on=['county_name'], right_on=['Area Name'])

polling_site_precinct_2016 = merged_2016[['id', 'Area Name', 'precinct_id']]

polling_site_precinct_2016["precinct_id"]= polling_site_precinct_2016["precinct_id"].astype(str)
polling_site_precinct_2016['precinct_id'] = polling_site_precinct_2016['precinct_id'].str.zfill(6)

polling_site_precinct_2016['Geographic Id'] = polling_site_precinct_2016['id'] + polling_site_precinct_2016['precinct_id']

polling_site_precinct_2016['Geographic Id'] = '7000000' + polling_site_precinct_2016['Geographic Id'].astype(str)

polling_site_precinct_2016 = polling_site_precinct_2016[[ 'Geographic Id', 'Area Name']]
polling_site_precinct_2016.rename(columns = {'Geographic Id': 'Id','Area Name': 'County'}, inplace = True)

polling_site_precinct_2016.to_csv('data/polling_site_data/2016_clean_precincts.csv')

In [506]:
print(polling_site_precinct_2016.shape)

(2696, 2)


In [507]:
polling_site_precinct_binary_2020 =  pd.read_csv('data/polling_site_data/2020_clean_precincts.csv', index_col = 0)
polling_site_precinct_binary_2016 =  pd.read_csv('data/polling_site_data/2016_clean_precincts.csv', index_col = 0)

Since these dataframes are the precincts that have a polling site, we insert a new column with just 1's. 

In [508]:
polling_site_precinct_binary_2020.insert(2, "Polling Site 2020", 1)
polling_site_precinct_binary_2016.insert(2, "Polling Site 2016", 1)

In [509]:
polling_site_precinct_binary_2016.head()

,Id,County,Polling Site 2016
0,7000000US1300100001B,Appling,1
1,7000000US1300100005A,Appling,1
2,7000000US1300100004D,Appling,1
3,7000000US1300100005C,Appling,1
4,7000000US1300100003C,Appling,1


In [510]:
polling_site_precinct_binary_2020.head()


,Id,County,Polling Site 2020
0,7000000US13001000002,Appling,1
1,7000000US1300100001C,Appling,1
2,7000000US1300100001B,Appling,1
3,7000000US1300100004D,Appling,1
4,7000000US1300100005A,Appling,1


By observing the dataframes to see the sizes, we can see the number of precincts in Georgia and the number of precincts that have polling sites in 2016 and 2020.

In [511]:
# Number of precincts in Georgia
print(race_precinct.shape)

# Number of polling sites in Georgia in 2016
print(polling_site_precinct_binary_2016.shape)

# Number of polling sites in Georgia in 2020
print(polling_site_precinct_binary_2020.shape)

(2698, 15)
(2696, 3)
(2678, 3)


Now we will combine these three dataframes to give a resulting dataframe with the racial densities of each precinct and whether it has a polling site in 2016 and 2020.

In [512]:
race_with_polling_site = race_precinct.merge(polling_site_precinct_binary_2016, how='left', left_on='id', right_on='Id')
race_with_polling_site.drop(['Id', 'County'], axis=1, inplace = True)


race_with_polling_site = race_with_polling_site.merge(polling_site_precinct_binary_2020, how='left', left_on='id', right_on='Id')
race_with_polling_site.drop(['Id', 'County'], axis=1, inplace = True)


In [513]:
race_with_polling_site.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density: Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others,Polling Site 2016,Polling Site 2020
0,"2, Appling County, Georgia",7000000US13001000002,3563.0,403.0,1215.0,1838.0,10.0,81.0,16.0,0.113107,0.341005,0.515857,0.002807,0.022734,0.004491,1.0,1.0
1,"1B, Appling County, Georgia",7000000US1300100001B,1834.0,76.0,1575.0,109.0,3.0,63.0,8.0,0.041439,0.858779,0.059433,0.001636,0.034351,0.004362,1.0,1.0
2,"1C, Appling County, Georgia",7000000US1300100001C,1538.0,116.0,1242.0,150.0,5.0,25.0,0.0,0.075423,0.807542,0.097529,0.003251,0.016255,0.000000,1.0,1.0
3,"3C, Appling County, Georgia",7000000US1300100003C,2515.0,263.0,1528.0,608.0,39.0,69.0,8.0,0.104573,0.607555,0.241750,0.015507,0.027435,0.003181,1.0,1.0
4,"4B, Appling County, Georgia",7000000US1300100004B,1321.0,62.0,1147.0,59.0,5.0,43.0,5.0,0.046934,0.868282,0.044663,0.003785,0.032551,0.003785,1.0,1.0


After the merge, some precincts have NaN in the 'Polling Site 2016' and 'Polling Site 2020', which means that they don't have a polling site. We will then replace NaN with '0'.

In [514]:
race_with_polling_site_isna = race_with_polling_site[race_with_polling_site.isna().any(axis=1)]
race_with_polling_site_isna.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density: Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others,Polling Site 2016,Polling Site 2020
51,"06, Barrow County, Georgia",7000000US13013000006,3656.0,334.0,2430.0,493.0,175.0,179.0,45.0,0.091357,0.664661,0.134847,0.047867,0.048961,0.012309,1.0,NaN
52,"07, Barrow County, Georgia",7000000US13013000007,4524.0,413.0,3084.0,700.0,114.0,196.0,17.0,0.091291,0.681698,0.154730,0.025199,0.043324,0.003758,1.0,NaN
54,"09, Barrow County, Georgia",7000000US13013000009,5165.0,698.0,3620.0,494.0,139.0,194.0,20.0,0.135140,0.700871,0.095644,0.026912,0.037561,0.003872,1.0,NaN
55,"10, Barrow County, Georgia",7000000US13013000010,5978.0,951.0,2796.0,1720.0,172.0,268.0,71.0,0.159083,0.467715,0.287722,0.028772,0.044831,0.011877,1.0,NaN
56,"11, Barrow County, Georgia",7000000US13013000011,7348.0,1377.0,4001.0,1310.0,256.0,332.0,72.0,0.187398,0.544502,0.178280,0.034839,0.045182,0.009799,1.0,NaN


In [515]:
race_with_polling_site.fillna(0, inplace = True)

Checking to make sure that there are no more null values.

In [516]:
race_with_polling_site.isna().sum()

Area Name                       0
id                              0
Total                           0
Hispanic                        0
White                           0
Black                           0
Asian                           0
Mixed                           0
Others                          0
Population Density: Hispanic    0
Population Density: White       0
Population Density: Black       0
Population Density: Asian       0
Population Density: Mixed       0
Population Density: Others      0
Polling Site 2016               0
Polling Site 2020               0
dtype: int64

Converting the binary numbers to integers.

In [517]:
race_with_polling_site['Polling Site 2016'] = race_with_polling_site['Polling Site 2016'].astype(int)
race_with_polling_site['Polling Site 2020'] = race_with_polling_site['Polling Site 2020'].astype(int)

In [518]:
race_with_polling_site.head()

,Area Name,id,Total,Hispanic,White,Black,Asian,Mixed,Others,Population Density: Hispanic,Population Density: White,Population Density: Black,Population Density: Asian,Population Density: Mixed,Population Density: Others,Polling Site 2016,Polling Site 2020
0,"2, Appling County, Georgia",7000000US13001000002,3563.0,403.0,1215.0,1838.0,10.0,81.0,16.0,0.113107,0.341005,0.515857,0.002807,0.022734,0.004491,1,1
1,"1B, Appling County, Georgia",7000000US1300100001B,1834.0,76.0,1575.0,109.0,3.0,63.0,8.0,0.041439,0.858779,0.059433,0.001636,0.034351,0.004362,1,1
2,"1C, Appling County, Georgia",7000000US1300100001C,1538.0,116.0,1242.0,150.0,5.0,25.0,0.0,0.075423,0.807542,0.097529,0.003251,0.016255,0.000000,1,1
3,"3C, Appling County, Georgia",7000000US1300100003C,2515.0,263.0,1528.0,608.0,39.0,69.0,8.0,0.104573,0.607555,0.241750,0.015507,0.027435,0.003181,1,1
4,"4B, Appling County, Georgia",7000000US1300100004B,1321.0,62.0,1147.0,59.0,5.0,43.0,5.0,0.046934,0.868282,0.044663,0.003785,0.032551,0.003785,1,1


In [519]:
race_with_polling_site.to_csv('data/polling_site_data/clean_precincts_with_polling_site.csv')